In [2]:
%pip install -r requirements-notebook.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import pipeline, SegformerFeatureExtractor, SegformerForSemanticSegmentation
from PIL import Image
import requests
import matplotlib.pyplot as plt
import numpy as np

# Load the model and feature extractor
feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b1-finetuned-cityscapes-1024-1024")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b1-finetuned-cityscapes-1024-1024")


/Users/tommyluu/Documents/projects/mf-nerf/venv/lib/python3.12/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/Users/tommyluu/Documents/projects/mf-nerf/venv/lib/python3.12/site-packages/transformers/models/segformer/image_processing_segformer.py:103: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [4]:
# Define the class names and color palette from Cityscapes
CLASSES = [
    "road", "sidewalk", "building", "wall", "fence", "pole", "traffic light", "traffic sign", 
    "vegetation", "terrain", "sky", "person", "rider", "car", "truck", "bus", "train", 
    "motorcycle", "bicycle"
]
PALETTE = np.array([[128, 64, 128], [244, 35, 232], [70, 70, 70], [102, 102, 156], [190, 153, 153], 
                    [153, 153, 153], [250, 170, 30], [220, 220, 0], [107, 142, 35], [152, 251, 152], 
                    [70, 130, 180], [220, 20, 60], [255, 0, 0], [0, 0, 142], [0, 0, 70], [0, 60, 100], 
                    [0, 80, 100], [0, 0, 230], [119, 11, 32]])


In [2]:
def load_image(image_path):
    """Load an image from a file or URL."""
    if image_path.startswith('http://') or image_path.startswith('https://'):
        return Image.open(requests.get(image_path, stream=True).raw).convert('RGB')
    else:
        return Image.open(image_path).convert('RGB')

In [35]:
def segment_image(image_path, output_path='output.png'):
    """Segment the image at the given path."""
    image = load_image(image_path)
    original_size = image.size  # Store original image size

    semantic_segmentation = pipeline("image-segmentation", "nvidia/segformer-b1-finetuned-cityscapes-1024-1024")
    results = semantic_segmentation(image)
    mask = np.zeros((original_size[1], original_size[0]), dtype=bool)
    for r in results:
        if r['label'] in ['car', 'truck', 'bus', 'train', 'motorcycle', 'bicycle', 'person', 'rider']: # to remove
            mask = mask | r['mask'] > 0

    # Apply transparency based on segmentation
    alpha_channel = np.where(mask, 0, 255).astype(np.uint8)  # Set alpha to 0 for masked areas
    image_rgba = image.convert("RGBA")
    image_array = np.array(image_rgba)
    image_array[:, :, 3] = alpha_channel  # Set alpha channel based on the mask
    
    # Save the modified image
    result_image = Image.fromarray(image_array)
    result_image.save(output_path)


In [36]:
from pathlib import Path

input_folder = Path('./input')
output_folder = Path('./output')
output_folder.mkdir(exist_ok=True)
for image_path in input_folder.glob('*.png'):
    output_path = output_folder / image_path.name
    segment_image(image_path.as_posix(), output_path.as_posix())



Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
